*Practical Data Science 19/20*
# Programming Assignment

In this programming assignment you need to apply your new `numpy`, `pandas` and `matplotlib` knowledge. You will need to do several [`groupby`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html)s and [`join`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.join.html)`s to solve the task. 

Load required packages

In [0]:
import pandas as pd
%matplotlib inline 

Load Data

In [0]:
DATA_URL = 'https://raw.githubusercontent.com/pds1920/_a1-template/master/data/'

transactions    = pd.read_csv(DATA_URL + '/sales_train.csv.gz')
items           = pd.read_csv(DATA_URL + '/items.csv')
item_categories = pd.read_csv(DATA_URL + '/item_categories.csv')

## Get to know the data
Print the **shape** of the loaded dataframes.
- You can use a list comprehension here

In [4]:
new_list = [transactions, items, item_categories]
for i in new_list:
    print(i.shape)

(1400001, 6)
(22170, 3)
(84, 2)


Use [`df.head`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.head.html) function to print several rows of each data frame.  Examine the features you are given.

In [5]:
# Write your code here
transactions.head(5)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,14.10.2013,9,19,7204,199.0,1.0
1,23.10.2013,9,19,7096,799.0,1.0
2,19.10.2013,9,19,7204,199.0,1.0
3,17.10.2013,9,19,7206,199.0,1.0
4,13.10.2013,9,19,7459,299.0,1.0


In [6]:
# Write your code here
items.head(5)

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [7]:
# Write your code here
item_categories.head(5)

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


Now use your `pandas` skills to get answers for the following questions. 

## What was the maximum total revenue among all the shops in June, 2014?


* Revenue refers to total sales minus value of goods returned.
* Sometimes items are returned, find such examples in the dataset. 
* It is handy to split `date` field into [`day`, `month`, `year`] components and use `df.year == 14` and `df.month == 6` in order to select target subset of dates.
* You may work with `date` feature as with strings, or you may first convert it to `pd.datetime` type with `pd.to_datetime` function, but do not forget to set correct `format` argument.

In [0]:
# Write your code here
transactions['date'] = pd.to_datetime(transactions['date'], format='%d.%m.%Y')

transactions['year'] = transactions['date'].dt.year
transactions['month'] = transactions['date'].dt.month
transactions['day'] = transactions['date'].dt.day

In [0]:
max_revenue = transactions[(transactions['year'] == 2014) & (transactions['month'] == 6)]
max_revenue = (transactions['item_price'] * transactions['item_cnt_day'])

max_revenue = max(max_revenue)

In [10]:
max_revenue

1829990.0000013

## How many items are there?

* Let's assume, that the items are returned for the same price as they had been sold

In [11]:
#num_items_constant_price =

ValueError: ignored

In [0]:
num_items_constant_price

## What was the variance of the number of sold items per day sequence for the shop with `shop_id = 25` in December, 2014?

* Do not count the items that were sold but returned back later.
* Fill `total_num_items_sold`: An (ordered) array that contains the total number of items sold on each day 
* Fill `days`: An (ordered) array that contains all relevant days
* Then compute variance of the of `total_num_items_sold`
* If there were no sales at a given day, ***do not*** impute missing value with zero, just ignore that day

In [0]:
shop_id = 25
sales_25 = transactions[(transactions['shop_id'] == shop_id) &
                        (transactions['year'] == 2014) &
                        (transactions['month'] == 12)]

In [0]:
# remove returned items
filtered_sales = sales_25.drop_duplicates(subset = 'item_id', keep = False)

# number of items sold on each day
total_num_items_sold = filtered_sales.groupby('day')['item_cnt_day'].sum()
total_num_items_sold_var = total_num_items_sold.var()

In [0]:
total_num_items_sold_var

## Vizualization of the daily items sold

Use `total_num_items_sold` and `days` arrays to and plot the daily revenue of `shop_id = 25` in December, 2014.

* plot-title: 'Daily items sold for shop_id = 25'

In [0]:
import numpy as np
total_num_items_sold.plot(kind='bar', x='day', title='Daily items sold for shop_id = 25');



## What item category that generated the highest revenue in spring 2014?</b></li>

   
Spring is the period from March to Mai.

In [0]:
spring_data = transactions[(transactions['year'] == 2014) &
                           (transactions['month'] >= 3) &
                           (transactions['month'] <= 5)]


spring_data.loc[:, 'revenue'] = spring_data['item_price'] * spring_data['item_cnt_day'].copy()


item_categories.rename(columns = {'item_category_id': 'item_id'}, inplace = True)
data_combined = pd.merge(spring_data, item_categories, on = 'item_id', how = 'left')

In [0]:
# group by item
category_id_with_max_revenue = data_combined.groupby(['item_id', 'item_category_name'])['revenue'].sum().sort_values(ascending=False).head(1)

In [0]:
category_id_with_max_revenue